# Analysis of the misalignment between non-COVID reported deaths and those estimated from the past*
<p style="text-align:right;font-style: italic;">*see Notes in the present document</p>

<h2 align="center">Theory</h2> 
During the COVID-19 pandemy in Italy, considering a generic month:
$$\text{monthly reported deaths} = (\text{non-COVID reported deaths}) + (\text{COVID-19 reported deaths})$$

<h3 align="center">Test</h3>
For <u>all the considered months</u> the following relationship should hold:
$$\text{non-COVID reported deaths}\ (month) \approx \overline{\text{non-COVID reported deaths}}\ (month)$$
where
$$\text{non-COVID reported deaths}\ (month) = (\text{monthly reported deaths}) - (\text{COVID-19 reported deaths})$$
and
$$\overline{\text{non-COVID reported deaths}}\ (month) = \frac{1}{N}\sum_{year < 2020}\text{monthly reported deaths }(month, year)$$
where $\text{monthly reported deaths}\ (month, year)$ are the reported deaths for one specific combination of month and year that follow the conditions: $year <2020$ and $month$ refers to one of the months interested by the pandemy in 2020.

<h3 align="center">Hypothesis</h3>

1. $\text{non-COVID reported deaths}$ can be estimated/inferred from the past (before the pandemy) returning $\overline{\text{non-COVID reported deaths}}$ and comparing it with the value computed by subtracting $\text{COVID-19 reported deaths}$ to $\text{monthly reported deaths}$;
2. $\text{COVID-19 reported deaths}$ refer only to deaths that are mainly caused by COVID-19 (individuals who were not in a serious clinical situation before contracting the virus);
3. all the $\text{monthly reported deaths}$, $\text{non-COVID reported deaths}$ and $\text{COVID-19 reported deaths}$ have been reported accurately and in the same month the individuals died.

**Comments**

Hypothesis `1.` seems reasonable if you are given a sufficiently long time series.

Hypothesis `2.` can be tested given the countings.

Hypothesis `3.`, with some approximations for the reportings at the beginning and the end of the month, should hold for $\text{non-COVID reported deaths}$.
The $\text{COVID-19 reported deaths}$ data may be less accurate (e.g. deaths in retirement homes discovered weeks late).

<h3 align="center">Data</h3>

Approximately from *March 1st 2020* to *November 30th 2020*:
- $\text{COVID-19 reported deaths}$: from <a url="https://github.com/pcm-dpc/COVID-19"><u>GitHub/pcm-dpc/COVID-19</u></a> repository maintained by the Protezione Civile Italiana (which is the italian Civil Protection).

Approximately from *January 1st 2015* to *September 30th 2020*:
- $\text{monthly reported deaths}$: from <a url="https://www.istat.it/"><u>ISTAT</u></a> database (ISTAT is the italian "National Institute of Statistics").

<h3 align="center">Notes</h3>

**The following document has the sole purpose of showing how to develop a scientific project through Python and how to use and manage tables with the `pandas` library; it has no claim to carry out scientific and/or medical results.**

In [6]:
import os
import pandas as pd

In [7]:
COVID_INPUT_PATH = os.path.join('data', 'pcm-dpc')
ISTAT_INPUT_PATH = os.path.join('data', 'istat')

In [8]:
COVID_INPUT_PATH

'data\\pcm-dpc'

## Deaths by COVID-19

In [9]:
%%time
covid_deaths_df = pd.read_csv(os.path.join(COVID_INPUT_PATH, 'covid19_deaths_per_region.csv'))

Wall time: 40.9 ms


In [14]:
covid_deaths_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6258 entries, 0 to 6257
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   date    6258 non-null   datetime64[ns]
 1   region  6258 non-null   object        
 2   deaths  6258 non-null   int64         
dtypes: datetime64[ns](1), int64(1), object(1)
memory usage: 146.8+ KB


In [15]:
covid_deaths_df.head(2)

,date,region,deaths
0,2020-02-24 18:00:00,Abruzzo,0
1,2020-02-24 18:00:00,Basilicata,0


In [17]:
covid_deaths_df.tail()

,date,region,deaths
6253,2020-12-17,Sicilia,2087
6254,2020-12-17,Toscana,3348
6255,2020-12-17,Umbria,549
6256,2020-12-17,Valle d'Aosta,362
6257,2020-12-17,Veneto,5161


### Field: `date`

The reporting hour is misleading because of DST to winter time change (and vice versa).

**Sunday, March 29th 2020**:    standard time -> daylight saving time \
**Sunday, October 25th 2020**:  daylight saving time -> standard time

In [16]:
# Series.dt.normalize() function to convert the times to midnight

covid_deaths_df['date'] = pd.to_datetime(covid_deaths_df['date']).dt.normalize()

In [18]:
covid_deaths_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6258 entries, 0 to 6257
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   date    6258 non-null   datetime64[ns]
 1   region  6258 non-null   object        
 2   deaths  6258 non-null   int64         
dtypes: datetime64[ns](1), int64(1), object(1)
memory usage: 146.8+ KB


In [19]:
covid_deaths_df['date'].max(), covid_deaths_df['date'].min()

(Timestamp('2020-12-17 00:00:00'), Timestamp('2020-02-24 00:00:00'))

In [20]:
print('Days between the start and the end of the recordings: ', covid_deaths_df.date.max() - covid_deaths_df.date.min())
print('Days of recording: ', len(covid_deaths_df.date.unique()))

Days between the start and the end of the recordings:  297 days 00:00:00
Days of recording:  298


### Field: `region`

**Italy is divided in 20 regions:**

<img src="imgs/italy_regions-wanderingitaly.com.png" width="400px">

In [27]:
regions = covid_deaths_df.region.unique().tolist()

In [28]:
regions

['Abruzzo',
 'Basilicata',
 'Calabria',
 'Campania',
 'Emilia-Romagna',
 'Friuli Venezia Giulia',
 'Lazio',
 'Liguria',
 'Lombardia',
 'Marche',
 'Molise',
 'Trentino-Alto Adige',
 'Piemonte',
 'Puglia',
 'Sardegna',
 'Sicilia',
 'Toscana',
 'Umbria',
 "Valle d'Aosta",
 'Veneto']

In [25]:
len(regions)

21

In [26]:
autonomous_province_to_region = {
    'P.A. Bolzano': 'Trentino-Alto Adige',
    'P.A. Trento': 'Trentino-Alto Adige',
}

covid_deaths_df.replace(to_replace={
    'region': autonomous_province_to_region
}, inplace=True)

In [35]:
covid_deaths_df = covid_deaths_df.groupby(['date', 'region']).deaths.sum().reset_index()

In [36]:
covid_deaths_df

,date,region,deaths
0,2020-02-24,Abruzzo,0
1,2020-02-24,Basilicata,0
2,2020-02-24,Calabria,0
3,2020-02-24,Campania,0
4,2020-02-24,Emilia-Romagna,0
...,...,...,...
5955,2020-12-17,Toscana,3348
5956,2020-12-17,Trentino-Alto Adige,1494
5957,2020-12-17,Umbria,549
5958,2020-12-17,Valle d'Aosta,362


We have a correct number of records since `298 days` times `20 regions` returns exactly `5960 rows`.

### Field: `deaths`

**The number of deaths is cumulative so, in order to deal with accurate data, given a day, the count must be equal or greater than the one reported the day before.**

In [39]:
cum_deaths_x_region = pd.pivot_table(data=covid_deaths_df, index=['date'], columns=['region'], values='deaths', aggfunc='sum')

In [40]:
cum_deaths_x_region.head(2)

region,Abruzzo,Basilicata,Calabria,Campania,Emilia-Romagna,Friuli Venezia Giulia,Lazio,Liguria,Lombardia,Marche,Molise,Piemonte,Puglia,Sardegna,Sicilia,Toscana,Trentino-Alto Adige,Umbria,Valle d'Aosta,Veneto
date,,,,,,,,,,,,,,,,,,,,
2020-02-24,0,0,0,0,0,0,0,0,6,0,0,0,0,0,0,0,0,0,0,1
2020-02-25,0,0,0,0,0,0,0,0,9,0,0,0,0,0,0,0,0,0,0,1


In [42]:
inc_deaths_x_region = cum_deaths_x_region.diff(periods=1)

In [43]:
inc_deaths_x_region

region,Abruzzo,Basilicata,Calabria,Campania,Emilia-Romagna,Friuli Venezia Giulia,Lazio,Liguria,Lombardia,Marche,Molise,Piemonte,Puglia,Sardegna,Sicilia,Toscana,Trentino-Alto Adige,Umbria,Valle d'Aosta,Veneto
date,,,,,,,,,,,,,,,,,,,,
2020-02-24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-02-25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2020-02-26,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2020-02-27,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2020-02-28,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-12-13,9.0,6.0,8.0,35.0,43.0,18.0,27.0,8.0,144.0,14.0,2.0,36.0,23.0,7.0,21.0,34.0,6.0,9.0,2.0,32.0
2020-12-14,19.0,5.0,4.0,30.0,73.0,14.0,38.0,17.0,67.0,10.0,1.0,48.0,24.0,11.0,32.0,42.0,25.0,5.0,0.0,26.0
2020-12-15,15.0,5.0,3.0,50.0,74.0,52.0,83.0,20.0,114.0,14.0,3.0,77.0,54.0,12.0,31.0,45.0,22.0,4.0,3.0,165.0


In [46]:
inc_deaths_x_region[inc_deaths_x_region < 0].dropna()

region,Abruzzo,Basilicata,Calabria,Campania,Emilia-Romagna,Friuli Venezia Giulia,Lazio,Liguria,Lombardia,Marche,Molise,Piemonte,Puglia,Sardegna,Sicilia,Toscana,Trentino-Alto Adige,Umbria,Valle d'Aosta,Veneto
date,,,,,,,,,,,,,,,,,,,,


**> The data seems consistent**

In [47]:
covid_deaths_df.deaths.describe()

count     5960.000000
mean      1608.860906
std       3521.396118
min          0.000000
25%         98.000000
50%        449.000000
75%       1217.250000
max      24165.000000
Name: deaths, dtype: float64

In [49]:
inc_deaths_x_region.describe().astype(int)

region,Abruzzo,Basilicata,Calabria,Campania,Emilia-Romagna,Friuli Venezia Giulia,Lazio,Liguria,Lombardia,Marche,Molise,Piemonte,Puglia,Sardegna,Sicilia,Toscana,Trentino-Alto Adige,Umbria,Valle d'Aosta,Veneto
count,297,297,297,297,297,297,297,297,297,297,297,297,297,297,297,297,297,297,297,297
mean,3,0,1,8,23,4,10,9,81,4,0,24,7,2,7,11,5,1,1,17
std,5,1,2,15,30,8,16,10,111,8,1,31,11,4,12,16,9,3,2,25
min,0,0,-1,0,0,0,0,0,0,-11,0,0,0,-1,0,0,-61,0,-1,-1
25%,0,0,0,0,1,0,1,0,3,0,0,1,0,0,0,0,0,0,0,1
50%,1,0,0,1,7,1,4,4,23,1,0,7,2,0,1,3,0,0,0,6
75%,6,0,1,8,40,5,12,15,129,8,0,45,8,3,6,17,9,1,2,25
max,35,16,17,85,155,52,83,51,546,56,8,120,72,49,49,75,49,21,12,165


### COVID-19 deaths per month

In [51]:
regions_cols = inc_deaths_x_region.columns

In [53]:
inc_deaths_x_region.reset_index(inplace=True)

In [54]:
inc_deaths_x_region.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 298 entries, 0 to 297
Data columns (total 21 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   date                   298 non-null    datetime64[ns]
 1   Abruzzo                297 non-null    float64       
 2   Basilicata             297 non-null    float64       
 3   Calabria               297 non-null    float64       
 4   Campania               297 non-null    float64       
 5   Emilia-Romagna         297 non-null    float64       
 6   Friuli Venezia Giulia  297 non-null    float64       
 7   Lazio                  297 non-null    float64       
 8   Liguria                297 non-null    float64       
 9   Lombardia              297 non-null    float64       
 10  Marche                 297 non-null    float64       
 11  Molise                 297 non-null    float64       
 12  Piemonte               297 non-null    float64       
 13  Pugli

In [55]:
monthly_covid_deaths_df = pd.DataFrame()
for col in regions_cols:
    tmp_df = pd.DataFrame({
        'date': inc_deaths_x_region['date'],
        'region': col,
        'covid_deaths_2020': inc_deaths_x_region[col],
    })
    monthly_covid_deaths_df = monthly_covid_deaths_df.append(tmp_df)

In [56]:
monthly_covid_deaths_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5960 entries, 0 to 297
Data columns (total 3 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   date               5960 non-null   datetime64[ns]
 1   region             5960 non-null   object        
 2   covid_deaths_2020  5940 non-null   float64       
dtypes: datetime64[ns](1), float64(1), object(1)
memory usage: 186.2+ KB


In [60]:
monthly_covid_deaths_df.tail()

,date,region,covid_deaths_2020,month
293,2020-12-13,Veneto,32.0,12
294,2020-12-14,Veneto,26.0,12
295,2020-12-15,Veneto,165.0,12
296,2020-12-16,Veneto,77.0,12
297,2020-12-17,Veneto,92.0,12


In [58]:
monthly_covid_deaths_df['month'] = monthly_covid_deaths_df.date.dt.month

In [62]:
monthly_covid_deaths_df = pd.pivot_table(
    data=monthly_covid_deaths_df,
    index=['month', 'region'],
    values='covid_deaths_2020',
    aggfunc='sum'
)

In [63]:
monthly_covid_deaths_df

covid_deaths_2020
month region                                
2     Abruzzo                            0.0
      Basilicata                         0.0
      Calabria                           0.0
      Campania                           0.0
      Emilia-Romagna                     4.0
...                                      ...
12    Toscana                          707.0
      Trentino-Alto Adige              309.0
      Umbria                           142.0
      Valle d'Aosta                     47.0
      Veneto                          1450.0

[220 rows x 1 columns]

In [64]:
mask1 = inc_deaths_x_region.date.dt.month == 2
inc_deaths_x_region.loc[mask1, ['date', 'Emilia-Romagna']]

region,date,Emilia-Romagna
0,2020-02-24,NaN
1,2020-02-25,0.0
2,2020-02-26,1.0
3,2020-02-27,0.0
4,2020-02-28,1.0
5,2020-02-29,2.0


**> The results are consistent**

## Historical deaths

In [65]:
%%time
hist_deaths_df = pd.read_csv(os.path.join(ISTAT_INPUT_PATH, 'decessi_comuni_giornaliero_30settembre.csv'))

Wall time: 9.15 s


In [66]:
hist_deaths_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2532662 entries, 0 to 2532661
Data columns (total 27 columns):
 #   Column          Dtype 
---  ------          ----- 
 0   REG             int64 
 1   PROV            int64 
 2   NOME_REGIONE    object
 3   NOME_PROVINCIA  object
 4   NOME_COMUNE     object
 5   COD_PROVCOM     int64 
 6   TIPO_COMUNE     int64 
 7   CL_ETA          int64 
 8   GE              int64 
 9   M_15            int64 
 10  M_16            int64 
 11  M_17            int64 
 12  M_18            int64 
 13  M_19            int64 
 14  M_20            object
 15  F_15            int64 
 16  F_16            int64 
 17  F_17            int64 
 18  F_18            int64 
 19  F_19            int64 
 20  F_20            object
 21  T_15            int64 
 22  T_16            int64 
 23  T_17            int64 
 24  T_18            int64 
 25  T_19            int64 
 26  T_20            object
dtypes: int64(21), object(6)
memory usage: 521.7+ MB


In [67]:
hist_deaths_df.head()

,REG,PROV,NOME_REGIONE,NOME_PROVINCIA,NOME_COMUNE,COD_PROVCOM,TIPO_COMUNE,CL_ETA,GE,M_15,...,F_17,F_18,F_19,F_20,T_15,T_16,T_17,T_18,T_19,T_20
0,11,41,Marche,Pesaro e Urbino,Acqualagna,41001,1,10,822,0,...,0,0,0,0,0,0,0,0,0,1
1,11,41,Marche,Pesaro e Urbino,Acqualagna,41001,1,10,1225,0,...,0,0,0,n.d.,0,1,0,0,0,n.d.
2,11,41,Marche,Pesaro e Urbino,Acqualagna,41001,1,11,629,0,...,0,0,0,0,0,1,0,0,0,0
3,11,41,Marche,Pesaro e Urbino,Acqualagna,41001,1,11,817,0,...,0,0,0,0,0,0,0,1,0,0
4,11,41,Marche,Pesaro e Urbino,Acqualagna,41001,1,11,923,0,...,0,0,0,0,0,0,1,0,0,0


In [68]:
hist_deaths_df.replace({'n.d.': '0'}, inplace=True)
hist_deaths_df.T_20 = hist_deaths_df.T_20.astype(int)

In [69]:
hist_deaths_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2532662 entries, 0 to 2532661
Data columns (total 27 columns):
 #   Column          Dtype 
---  ------          ----- 
 0   REG             int64 
 1   PROV            int64 
 2   NOME_REGIONE    object
 3   NOME_PROVINCIA  object
 4   NOME_COMUNE     object
 5   COD_PROVCOM     int64 
 6   TIPO_COMUNE     int64 
 7   CL_ETA          int64 
 8   GE              int64 
 9   M_15            int64 
 10  M_16            int64 
 11  M_17            int64 
 12  M_18            int64 
 13  M_19            int64 
 14  M_20            object
 15  F_15            int64 
 16  F_16            int64 
 17  F_17            int64 
 18  F_18            int64 
 19  F_19            int64 
 20  F_20            object
 21  T_15            int64 
 22  T_16            int64 
 23  T_17            int64 
 24  T_18            int64 
 25  T_19            int64 
 26  T_20            int32 
dtypes: int32(1), int64(21), object(5)
memory usage: 512.1+ MB


**Adjustments:**
- Since the COVID-19 data is aggregated by region, in this database we do not have to consider the town and the province.
- Since the COVID-19 data is not declined by gender, in this database we only care about the `total_deaths`: we will not consider the columns `M_15`, `M_16`, ..., `F_20`.
- Since the COVID-19 data is not declined by age, in this database we do not have to consider the age class (column `CL_ETA`).

### Field: `region`

In [70]:
cols_to_rename = {
    'NOME_REGIONE': 'region',
}

hist_deaths_df.rename(columns=cols_to_rename, inplace=True)

In [71]:
regions = hist_deaths_df.region.unique().tolist()

In [72]:
len(regions)

20

In [73]:
regions

['Marche',
 'Toscana',
 'Umbria',
 'Lazio',
 'Trentino-Alto Adige/Südtirol',
 'Veneto',
 'Friuli-Venezia Giulia',
 'Emilia-Romagna',
 'Piemonte',
 "Valle d'Aosta/Vallée d'Aoste",
 'Liguria',
 'Lombardia',
 'Campania',
 'Abruzzo',
 'Molise',
 'Puglia',
 'Basilicata',
 'Calabria',
 'Sicilia',
 'Sardegna']

In [74]:
hist_deaths_df

,REG,PROV,region,NOME_PROVINCIA,NOME_COMUNE,COD_PROVCOM,TIPO_COMUNE,CL_ETA,GE,M_15,...,F_17,F_18,F_19,F_20,T_15,T_16,T_17,T_18,T_19,T_20
0,11,41,Marche,Pesaro e Urbino,Acqualagna,41001,1,10,822,0,...,0,0,0,0,0,0,0,0,0,1
1,11,41,Marche,Pesaro e Urbino,Acqualagna,41001,1,10,1225,0,...,0,0,0,0,0,1,0,0,0,0
2,11,41,Marche,Pesaro e Urbino,Acqualagna,41001,1,11,629,0,...,0,0,0,0,0,1,0,0,0,0
3,11,41,Marche,Pesaro e Urbino,Acqualagna,41001,1,11,817,0,...,0,0,0,0,0,0,0,1,0,0
4,11,41,Marche,Pesaro e Urbino,Acqualagna,41001,1,11,923,0,...,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2532657,20,111,Sardegna,Sud Sardegna,Villaspeciosa,111107,1,20,408,0,...,0,0,0,1,0,0,0,0,0,1
2532658,20,111,Sardegna,Sud Sardegna,Villaspeciosa,111107,1,20,907,0,...,0,0,0,0,0,1,0,0,0,0
2532659,20,111,Sardegna,Sud Sardegna,Villaspeciosa,111107,1,20,1029,0,...,0,0,0,0,0,0,0,0,1,0
2532660,20,111,Sardegna,Sud Sardegna,Villaspeciosa,111107,1,20,1113,0,...,1,0,0,0,0,0,1,0,0,0


In [75]:
hist_deaths_df.region = hist_deaths_df.region.apply(lambda x: x.split('/')[0])

In [76]:
hist_deaths_df.region.unique().tolist()

['Marche',
 'Toscana',
 'Umbria',
 'Lazio',
 'Trentino-Alto Adige',
 'Veneto',
 'Friuli-Venezia Giulia',
 'Emilia-Romagna',
 'Piemonte',
 "Valle d'Aosta",
 'Liguria',
 'Lombardia',
 'Campania',
 'Abruzzo',
 'Molise',
 'Puglia',
 'Basilicata',
 'Calabria',
 'Sicilia',
 'Sardegna']

### Field: `date`

Since the database is already very big, let's reduce it by considering only the month and not even the day.

In [78]:
# Compose the year-month field
def extract_month(string):
    month = str(string)[:-2]
    return month

hist_deaths_df['month'] = hist_deaths_df.GE.apply(extract_month).astype(int)

In [79]:
hist_deaths_df.head(2)

,REG,PROV,region,NOME_PROVINCIA,NOME_COMUNE,COD_PROVCOM,TIPO_COMUNE,CL_ETA,GE,M_15,...,F_18,F_19,F_20,T_15,T_16,T_17,T_18,T_19,T_20,month
0,11,41,Marche,Pesaro e Urbino,Acqualagna,41001,1,10,822,0,...,0,0,0,0,0,0,0,0,1,8
1,11,41,Marche,Pesaro e Urbino,Acqualagna,41001,1,10,1225,0,...,0,0,0,0,1,0,0,0,0,12


In [82]:
deaths_cols = [f'T_{y}' for y in range(15, 21)]

monthly_hist_deaths_df = pd.pivot_table(data=hist_deaths_df, index=['region', 'month'], values=deaths_cols, aggfunc='sum').reset_index()

In [83]:
monthly_hist_deaths_df.head()

,region,month,T_15,T_16,T_17,T_18,T_19,T_20
0,Abruzzo,1,1572,1431,1978,1672,1537,1459
1,Abruzzo,2,1468,1242,1337,1348,1402,1362
2,Abruzzo,3,1431,1327,1347,1409,1347,1566
3,Abruzzo,4,1250,1218,1220,1224,1223,1431
4,Abruzzo,5,1189,1244,1225,1141,1154,1208


In [84]:
monthly_hist_deaths_df.query('region == "Veneto"')

,region,month,T_15,T_16,T_17,T_18,T_19,T_20
228,Veneto,1,5398,4609,5699,4973,4929,4792
229,Veneto,2,4458,4335,4452,4338,4697,4336
230,Veneto,3,4528,4445,4431,4572,4311,5426
231,Veneto,4,4092,3824,3970,4045,3909,5184
232,Veneto,5,3895,4016,3978,3858,3930,4110
233,Veneto,6,3592,3723,3660,3604,3920,3780
234,Veneto,7,4163,3725,3659,3661,3827,3893
235,Veneto,8,3910,3814,3846,4089,3770,4067
236,Veneto,9,3627,3582,3576,3774,3745,3862
237,Veneto,10,4065,3956,4211,4192,4082,0


**We can remove October, November and December since no records are provided for 2020.**

In [85]:
mask = monthly_hist_deaths_df.month.isin([10, 11, 12])
monthly_hist_deaths_df.drop(
    index=monthly_hist_deaths_df[mask].index,
    inplace=True
)

In [86]:
monthly_hist_deaths_df.query('region == "Veneto"')

,region,month,T_15,T_16,T_17,T_18,T_19,T_20
228,Veneto,1,5398,4609,5699,4973,4929,4792
229,Veneto,2,4458,4335,4452,4338,4697,4336
230,Veneto,3,4528,4445,4431,4572,4311,5426
231,Veneto,4,4092,3824,3970,4045,3909,5184
232,Veneto,5,3895,4016,3978,3858,3930,4110
233,Veneto,6,3592,3723,3660,3604,3920,3780
234,Veneto,7,4163,3725,3659,3661,3827,3893
235,Veneto,8,3910,3814,3846,4089,3770,4067
236,Veneto,9,3627,3582,3576,3774,3745,3862


In [87]:
avg_deaths_cols = [f'T_{y}' for y in range(15, 20)]
monthly_hist_deaths_df['avg_deaths'] = monthly_hist_deaths_df[avg_deaths_cols].mean(axis=1).round(1)
monthly_hist_deaths_df['avg_stddev'] = monthly_hist_deaths_df[avg_deaths_cols].std(axis=1).round(1)

monthly_hist_deaths_df.rename(columns={'T_20': 'tot_deaths_2020'}, inplace=True)

In [88]:
monthly_hist_deaths_df

,region,month,T_15,T_16,T_17,T_18,T_19,tot_deaths_2020,avg_deaths,avg_stddev
0,Abruzzo,1,1572,1431,1978,1672,1537,1459,1638.0,208.7
1,Abruzzo,2,1468,1242,1337,1348,1402,1362,1359.4,83.7
2,Abruzzo,3,1431,1327,1347,1409,1347,1566,1372.2,45.1
3,Abruzzo,4,1250,1218,1220,1224,1223,1431,1227.0,13.1
4,Abruzzo,5,1189,1244,1225,1141,1154,1208,1190.6,44.3
...,...,...,...,...,...,...,...,...,...,...
232,Veneto,5,3895,4016,3978,3858,3930,4110,3935.4,63.2
233,Veneto,6,3592,3723,3660,3604,3920,3780,3699.8,133.6
234,Veneto,7,4163,3725,3659,3661,3827,3893,3807.0,210.4
235,Veneto,8,3910,3814,3846,4089,3770,4067,3885.8,124.5


In [89]:
monthly_hist_deaths_df.drop(columns=avg_deaths_cols, inplace=True)
monthly_hist_deaths_df.sort_values(['month', 'region'], inplace=True)
monthly_hist_deaths_df.set_index(['month', 'region'], inplace=True)

In [90]:
monthly_hist_deaths_df

tot_deaths_2020  avg_deaths  avg_stddev
month region                                                      
1     Abruzzo                         1459      1638.0       208.7
      Basilicata                       564       663.4        69.9
      Calabria                        2009      2239.6       270.5
      Campania                        5596      6110.4       678.1
      Emilia-Romagna                  4693      5255.8       546.2
...                                    ...         ...         ...
9     Toscana                         3461      3201.2        65.0
      Trentino-Alto Adige              723       710.4        23.2
      Umbria                           779       749.8        20.3
      Valle d'Aosta                    103       106.4         5.5
      Veneto                          3862      3660.8        92.8

[180 rows x 3 columns]

## Deaths comparison

In [92]:
monthly_covid_deaths_df, monthly_hist_deaths_df

(                           covid_deaths_2020
 month region                                
 2     Abruzzo                            0.0
       Basilicata                         0.0
       Calabria                           0.0
       Campania                           0.0
       Emilia-Romagna                     4.0
 ...                                      ...
 12    Toscana                          707.0
       Trentino-Alto Adige              309.0
       Umbria                           142.0
       Valle d'Aosta                     47.0
       Veneto                          1450.0
 
 [220 rows x 1 columns],
                            tot_deaths_2020  avg_deaths  avg_stddev
 month region                                                      
 1     Abruzzo                         1459      1638.0       208.7
       Basilicata                       564       663.4        69.9
       Calabria                        2009      2239.6       270.5
       Campania                    

In [94]:
df = monthly_covid_deaths_df.merge(
    monthly_hist_deaths_df,
    left_index=True,
    right_index=True
)

In [95]:
df.head()

covid_deaths_2020  tot_deaths_2020  avg_deaths  \
month region                                                           
2     Abruzzo                       0.0             1362      1359.4   
      Basilicata                    0.0              602       581.6   
      Calabria                      0.0             1732      1858.6   
      Campania                      0.0             4884      4970.8   
      Emilia-Romagna                4.0             4384      4437.0   

                      avg_stddev  
month region                      
2     Abruzzo               83.7  
      Basilicata            52.3  
      Calabria             125.9  
      Campania             328.2  
      Emilia-Romagna       233.0

In [96]:
df['noncovid_deaths_2020'] = df.tot_deaths_2020 - df.covid_deaths_2020

In [97]:
df.tail()

covid_deaths_2020  tot_deaths_2020  avg_deaths  \
month region                                                                
9     Toscana                           23.0             3461      3201.2   
      Trentino-Alto Adige                1.0              723       710.4   
      Umbria                             5.0              779       749.8   
      Valle d'Aosta                      0.0              103       106.4   
      Veneto                            58.0             3862      3660.8   

                           avg_stddev  noncovid_deaths_2020  
month region                                                 
9     Toscana                    65.0                3438.0  
      Trentino-Alto Adige        23.2                 722.0  
      Umbria                     20.3                 774.0  
      Valle d'Aosta               5.5                 103.0  
      Veneto                     92.8                3804.0

In [98]:
factor = 2
df['sup_value_acceptance'] = df.avg_deaths + factor*df.avg_stddev
df['inf_value_acceptance'] = df.avg_deaths - factor*df.avg_stddev

In [99]:
df.tail()

covid_deaths_2020  tot_deaths_2020  avg_deaths  \
month region                                                                
9     Toscana                           23.0             3461      3201.2   
      Trentino-Alto Adige                1.0              723       710.4   
      Umbria                             5.0              779       749.8   
      Valle d'Aosta                      0.0              103       106.4   
      Veneto                            58.0             3862      3660.8   

                           avg_stddev  noncovid_deaths_2020  \
month region                                                  
9     Toscana                    65.0                3438.0   
      Trentino-Alto Adige        23.2                 722.0   
      Umbria                     20.3                 774.0   
      Valle d'Aosta               5.5                 103.0   
      Veneto                     92.8                3804.0   

                           sup_value_acceptance  inf_value_acceptance  
month region                                                           
9     Toscana                            3331.2                3071.2  
      Trentino-Alto Adige                 756.8                 664.0  
      Umbria                              790.4                 709.2  
      Valle d'Aosta                       117.4                  95.4  
      Veneto                             3846.4                3475.2

In [100]:
def eval_approx(val, min_val, max_val):
    if min_val <= val <= max_val:
        return 1
    else:
        return 0

df['approx_is_valid'] = df.apply(lambda x: 
                                 eval_approx(val=x['noncovid_deaths_2020'], min_val=x['inf_value_acceptance'], max_val=x['sup_value_acceptance']),
                                 axis=1)

In [101]:
df

covid_deaths_2020  tot_deaths_2020  avg_deaths  \
month region                                                                
2     Abruzzo                            0.0             1362      1359.4   
      Basilicata                         0.0              602       581.6   
      Calabria                           0.0             1732      1858.6   
      Campania                           0.0             4884      4970.8   
      Emilia-Romagna                     4.0             4384      4437.0   
...                                      ...              ...         ...   
9     Toscana                           23.0             3461      3201.2   
      Trentino-Alto Adige                1.0              723       710.4   
      Umbria                             5.0              779       749.8   
      Valle d'Aosta                      0.0              103       106.4   
      Veneto                            58.0             3862      3660.8   

                           avg_stddev  noncovid_deaths_2020  \
month region                                                  
2     Abruzzo                    83.7                1362.0   
      Basilicata                 52.3                 602.0   
      Calabria                  125.9                1732.0   
      Campania                  328.2                4884.0   
      Emilia-Romagna            233.0                4380.0   
...                               ...                   ...   
9     Toscana                    65.0                3438.0   
      Trentino-Alto Adige        23.2                 722.0   
      Umbria                     20.3                 774.0   
      Valle d'Aosta               5.5                 103.0   
      Veneto                     92.8                3804.0   

                           sup_value_acceptance  inf_value_acceptance  \
month region                                                            
2     Abruzzo                            1526.8                1192.0   
      Basilicata                          686.2                 477.0   
      Calabria                           2110.4                1606.8   
      Campania                           5627.2                4314.4   
      Emilia-Romagna                     4903.0                3971.0   
...                                         ...                   ...   
9     Toscana                            3331.2                3071.2   
      Trentino-Alto Adige                 756.8                 664.0   
      Umbria                              790.4                 709.2   
      Valle d'Aosta                       117.4                  95.4   
      Veneto                             3846.4                3475.2   

                           approx_is_valid  
month region                                
2     Abruzzo                            1  
      Basilicata                         1  
      Calabria                           1  
      Campania                           1  
      Emilia-Romagna                     1  
...                                    ...  
9     Toscana                            0  
      Trentino-Alto Adige                1  
      Umbria                             1  
      Valle d'Aosta                      1  
      Veneto                             1  

[152 rows x 8 columns]

In [102]:
df.query('approx_is_valid == 0').round()

covid_deaths_2020  tot_deaths_2020  avg_deaths  \
month region                                                                
2     Sicilia                            0.0             4594      5016.0   
3     Emilia-Romagna                  1640.0             7761      4582.0   
      Liguria                          428.0             3038      1970.0   
      Lombardia                       7176.0            25622      8778.0   
      Marche                           452.0             2332      1620.0   
      Piemonte                         854.0             7261      4740.0   
      Puglia                           110.0             4043      3599.0   
      Trentino-Alto Adige              240.0             1418       872.0   
      Veneto                           475.0             5426      4457.0   
4     Emilia-Romagna                  1907.0             6149      4014.0   
      Liguria                          739.0             2862      1760.0   
      Lombardia                       6573.0            17212      7902.0   
      Piemonte                        2212.0             7435      4201.0   
      Puglia                           305.0             3748      3218.0   
      Trentino-Alto Adige              453.0             1335       772.0   
      Veneto                           982.0             5184      3968.0   
5     Campania                          53.0             4129      4358.0   
      Emilia-Romagna                   563.0             4171      4028.0   
      Lazio                            294.0             4497      4632.0   
      Lombardia                       2340.0             8873      7824.0   
      Marche                            81.0             1419      1426.0   
      Piemonte                         801.0             4640      4213.0   
      Toscana                          199.0             3325      3520.0   
      Veneto                           459.0             4110      3935.0   
6     Liguria                           93.0             1654      1707.0   
      Molise                             1.0              303       287.0   
      Piemonte                         223.0             3982      4108.0   
8     Trentino-Alto Adige                0.0              771       720.0   
      Valle d'Aosta                      0.0              130       107.0   
9     Liguria                           33.0             1783      1612.0   
      Sardegna                          20.0             1319      1196.0   
      Toscana                           23.0             3461      3201.0   

                           avg_stddev  noncovid_deaths_2020  \
month region                                                  
2     Sicilia                   199.0                4594.0   
3     Emilia-Romagna            217.0                6121.0   
      Liguria                   102.0                2610.0   
      Lombardia                 297.0               18446.0   
      Marche                     57.0                1880.0   
      Piemonte                  140.0                6407.0   
      Puglia                    154.0                3933.0   
      Trentino-Alto Adige        54.0                1178.0   
      Veneto                    100.0                4951.0   
4     Emilia-Romagna             75.0                4242.0   
      Liguria                    51.0                2123.0   
      Lombardia                 139.0               10639.0   
      Piemonte                  115.0                5223.0   
      Puglia                     86.0                3443.0   
      Trentino-Alto Adige        24.0                 882.0   
      Veneto                    107.0                4202.0   
5     Campania                   45.0                4076.0   
      Emilia-Romagna            121.0                3608.0   
      Lazio                     154.0                4203.0   
      Lombardia                 253.0                6533.0   
      Marche          